In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
%matplotlib inline

In [11]:
#df_P = pd.read_csv(filepath_or_buffer = 'D3_pre.csv', parse_dates = [[1,2]], index_col = False)  
#df_P = df_P.set_index('YYYY/MM/DD_HH:MM:SS')
#df_P.drop(df_P.columns[[0,2,3,4,5,6,9,10,11,12,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40]], axis=1, inplace=True)
df_P = pd.io.parsers.read_csv(filepath_or_buffer = 'D0_filt_10mino3.csv',index_col = 0)
print df_P[:5]

                       O3_ppb      UnixTime       e2v03       Temp         Rh  \
DATE (MM/DD/YYYY)_MST                                                           
2014-07-10 20:25:00        41  1.405024e+09  114.714286  45.800000  11.400000   
2014-07-10 20:26:00        39  1.405024e+09  120.428571  45.814286  11.314286   
2014-07-10 20:27:00        44  1.405024e+09  119.142857  45.857143  11.300000   
2014-07-10 20:28:00        47  1.405024e+09  117.500000  45.900000  11.300000   
2014-07-10 20:29:00        44  1.405024e+09  120.285714  45.900000  11.457143   

                       Zenith Angle [degrees]  days from start  ref_o3_smooth  \
DATE (MM/DD/YYYY)_MST                                                           
2014-07-10 20:25:00                  99.48960                0             41   
2014-07-10 20:26:00                  99.63789                0             39   
2014-07-10 20:27:00                  99.78580                0             44   
2014-07-10 20:28:00        

In [12]:
df_P.drop(df_P.columns[[0,2]], axis=1, inplace=True)
print df_P[:5]

                           UnixTime       Temp         Rh  \
DATE (MM/DD/YYYY)_MST                                       
2014-07-10 20:25:00    1.405024e+09  45.800000  11.400000   
2014-07-10 20:26:00    1.405024e+09  45.814286  11.314286   
2014-07-10 20:27:00    1.405024e+09  45.857143  11.300000   
2014-07-10 20:28:00    1.405024e+09  45.900000  11.300000   
2014-07-10 20:29:00    1.405024e+09  45.900000  11.457143   

                       Zenith Angle [degrees]  days from start  ref_o3_smooth  \
DATE (MM/DD/YYYY)_MST                                                           
2014-07-10 20:25:00                  99.48960                0             41   
2014-07-10 20:26:00                  99.63789                0             39   
2014-07-10 20:27:00                  99.78580                0             44   
2014-07-10 20:28:00                  99.93332                0             47   
2014-07-10 20:29:00                 100.08045                0             44   

    

####Define the slope function.

In [13]:
def find_mean_slope_over_time(df_P, int_min, data_col):
    df_P['diff'] = df_P[data_col].diff()
    
    slope = []
    for i in range(int_min,len(df_P['diff'])):
        top = i-int_min
        slope.append(df_P['diff'][top:i].mean())
    
    for i in range(0,int_min):
        slope.insert(i, 'NaN')

    return slope

####Define functions that create features that represent the area under the temp and humidity curves.

In [14]:
def make_time_integral_features(df_P,int_min, data_col):
    int_min = 90
    area_curve = []

    for i in range(int_min,len(df_P[data_col])):
        top = i-int_min
        area_curve.append(np.trapz(df_P[data_col][top:i]))
    
    for i in range(0,int_min):
        area_curve.insert(i, 'NaN')
    
    return area_curve

###Call the slope functions and add them to the dataframe.

In [15]:
df_P['o3_slope_90'] = find_mean_slope_over_time(df_P, 90,'pod_o3_smooth')
df_P['o3_slope_75'] = find_mean_slope_over_time(df_P, 75,'pod_o3_smooth')
df_P['o3_slope_60'] = find_mean_slope_over_time(df_P, 60,'pod_o3_smooth')
df_P['o3_slope_45'] = find_mean_slope_over_time(df_P, 45,'pod_o3_smooth')
df_P['o3_slope_30'] = find_mean_slope_over_time(df_P, 30,'pod_o3_smooth')
df_P['o3_slope_15'] = find_mean_slope_over_time(df_P, 15,'pod_o3_smooth')
df_P['o3_slope_5'] = find_mean_slope_over_time(df_P, 5,'pod_o3_smooth')

df_P['temp_slope_90'] = find_mean_slope_over_time(df_P, 90,'Temp')
df_P['temp_slope_75'] = find_mean_slope_over_time(df_P, 75,'Temp')
df_P['temp_slope_60'] = find_mean_slope_over_time(df_P, 60,'Temp')
df_P['temp_slope_45'] = find_mean_slope_over_time(df_P, 45,'Temp')
df_P['temp_slope_30'] = find_mean_slope_over_time(df_P, 30,'Temp')
df_P['temp_slope_15'] = find_mean_slope_over_time(df_P, 15,'Temp')
df_P['temp_slope_5'] = find_mean_slope_over_time(df_P, 5,'Temp')

df_P['rh_slope_90'] = find_mean_slope_over_time(df_P, 90,'Rh')
df_P['rh_slope_75'] = find_mean_slope_over_time(df_P, 75,'Rh')
df_P['rh_slope_60'] = find_mean_slope_over_time(df_P, 60,'Rh')
df_P['rh_slope_45'] = find_mean_slope_over_time(df_P, 45,'Rh')
df_P['rh_slope_30'] = find_mean_slope_over_time(df_P, 30,'Rh')
df_P['rh_slope_15'] = find_mean_slope_over_time(df_P, 15,'Rh')
df_P['rh_slope_5'] = find_mean_slope_over_time(df_P, 5,'Rh')

###Call the integration functions and add them to the dataframe.

In [16]:
#df_P['rh_int_105'] = make_time_integral_features(df_P,105,'Rh')
df_P['rh_int_90'] = make_time_integral_features(df_P,90,'Rh')
df_P['rh_int_75'] = make_time_integral_features(df_P,75,'Rh')
df_P['rh_int_60'] = make_time_integral_features(df_P,60,'Rh')
df_P['rh_int_45'] = make_time_integral_features(df_P,45,'Rh')
df_P['rh_int_30'] = make_time_integral_features(df_P,30,'Rh')

#df_P['temp_int_105'] = make_time_integral_features(df_P,105,'Temp')
df_P['temp_int_90'] = make_time_integral_features(df_P,90,'Temp')
df_P['temp_int_75'] = make_time_integral_features(df_P,75,'Temp')
df_P['temp_int_60'] = make_time_integral_features(df_P,60,'Temp')
df_P['temp_int_45'] = make_time_integral_features(df_P,45,'Temp')
df_P['temp_int_30'] = make_time_integral_features(df_P,30,'Temp')

df_P['o3_int_90'] = make_time_integral_features(df_P,90,'pod_o3_smooth')
df_P['o3_int_75'] = make_time_integral_features(df_P,75,'pod_o3_smooth')
df_P['o3_int_60'] = make_time_integral_features(df_P,60,'pod_o3_smooth')
df_P['o3_int_45'] = make_time_integral_features(df_P,45,'pod_o3_smooth')
df_P['o3_int_30'] = make_time_integral_features(df_P,30,'pod_o3_smooth')

In [17]:
#Delete the first 90 rows of the dataframe to remove all NaNs.
df_P = df_P.ix[90:]

In [18]:
#ozone terms
df_P['O3_sq'] = df_P['pod_o3_smooth']**2
df_P['O3_cu'] = df_P['pod_o3_smooth']**3
df_P['ln_O3'] = np.log(df_P['pod_o3_smooth'])

#temp terms
df_P['temp_sq'] = df_P['Temp']**2
df_P['temp_cu'] = df_P['Temp']**3
df_P['ln_temp'] = np.log(df_P['Temp'])
df_P['ln_temp_int_90'] = np.log(df_P['temp_int_90'].astype(int))

#rh terms
df_P['rh_sq'] = df_P['Rh']**2
df_P['rh_cu'] = df_P['Rh']**3
df_P['ln_rh'] = np.log(df_P['Rh'])
df_P['ln_rh_int_90'] = np.log(df_P['rh_int_90'].astype(int))

#ozone/temp interactions
df_P['03_mult_temp'] = df_P['pod_o3_smooth']*df_P['Temp']
df_P['03_mult_temp_sq'] = (df_P['pod_o3_smooth']*df_P['Temp'])**2
df_P['03_mult_temp_cu'] = (df_P['pod_o3_smooth']*df_P['Temp'])**3
df_P['ln_03_mult_temp'] = np.log(df_P['03_mult_temp'])

#ozone/rh interactions
df_P['03_mult_rh'] = df_P['pod_o3_smooth']*df_P['Rh']
df_P['03_mult_rh_sq'] = (df_P['pod_o3_smooth']*df_P['Rh'])**2
df_P['03_mult_rh_cu'] = (df_P['pod_o3_smooth']*df_P['Rh'])**3
df_P['ln_03_mult_rh'] = np.log(df_P['03_mult_rh'])

#temp/rh interactions
df_P['temp_mult_rh'] = df_P['Temp']*df_P['Rh']
df_P['temp_mult_rh_sq'] = (df_P['Temp']*df_P['Rh'])**2
df_P['temp_mult_rh_cu'] = (df_P['Temp']*df_P['Rh'])**3
df_P['ln_temp_mult_rh'] = np.log(df_P['temp_mult_rh'])


#ozone/rh/temp interactions
df_P['03_mult_rh_&_temp'] = df_P['pod_o3_smooth']*df_P['Rh']*df_P['Temp']
df_P['03_mult_rh_&_temp_sq'] = (df_P['pod_o3_smooth']*df_P['Rh']*df_P['Temp'])**2
df_P['03_mult_rh_&_temp_cu'] = (df_P['pod_o3_smooth']*df_P['Rh']*df_P['Temp'])**3
df_P['ln_03_mult_rh_&_temp_cu'] = np.log(df_P['03_mult_rh_&_temp_cu'])

print df_P[:5]

                           UnixTime       Temp         Rh  \
DATE (MM/DD/YYYY)_MST                                       
2014-07-10 21:55:00    1.405029e+09  43.200000  11.971429   
2014-07-10 21:56:00    1.405029e+09  43.200000  11.800000   
2014-07-10 21:57:00    1.405029e+09  43.214286  11.700000   
2014-07-10 21:58:00    1.405030e+09  43.257143  11.742857   
2014-07-10 21:59:00    1.405030e+09  43.228571  11.757143   

                       Zenith Angle [degrees]  days from start  ref_o3_smooth  \
DATE (MM/DD/YYYY)_MST                                                           
2014-07-10 21:55:00                 110.98962                0             52   
2014-07-10 21:56:00                 111.09270                0             55   
2014-07-10 21:57:00                 111.19515                0             59   
2014-07-10 21:58:00                 111.29696                0             56   
2014-07-10 21:59:00                 111.39814                0             51   

    

In [19]:
df_P.to_csv(path_or_buf = 'D0_smooth_features.csv')